In [37]:
import numpy as np
np.set_printoptions(edgeitems=10,linewidth=390)


In [261]:
def create_ball_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    # dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    dist_from_center = np.abs(X - center[0]) + np.abs(Y-center[1])

    mask = dist_from_center <= radius
    return mask


scans = []
sensor_locs = []
beacon_locs = []
'Sensor at x=2, y=18: closest beacon is at x=-2, y=15'
for i, scan_txt in enumerate(inputs.split('\n')):
    # if i != 6: continue
    sensor_loc, beacon_loc = [toks.split('at ')[1] for toks in scan_txt.split(':')]
    sensor_locs.append([int(v.split('=')[1]) for v in sensor_loc.split(', ')])
    beacon_locs.append([int(v.split('=')[1]) for v in beacon_loc.split(', ')])
sensor_locs = np.array(sensor_locs)
beacon_locs = np.array(beacon_locs)
detection_dists = np.abs((beacon_locs - sensor_locs)).sum(axis=-1)


locs = np.concatenate([sensor_locs, beacon_locs])
xmin, xmax = (sensor_locs[:,0] - detection_dists ).min(), (sensor_locs[:,0] + detection_dists + 0).max()
ymin, ymax = (sensor_locs[:,1] - detection_dists ).min(), (sensor_locs[:,1] + detection_dists + 0).max()

axis_mins = np.array([xmin, ymin])

# combined_map = np.zeros((xmax - xmin + 1, ymax - ymin + 1), np.int32)


# for i, scanner, beacon, dist in zip(range(len(sensor_locs)), sensor_locs, beacon_locs, detection_dists):
#     # if i >= 1: continue
#     idx_beacon = beacon - axis_mins
#     idx = scanner - axis_mins
#     start = tuple(idx - dist)
#     end = tuple(idx + dist + 1)
#     # print(start, end, dist, scanner)
#     h_lo, h_hi = max(start[0],0), min(end[0], combined_map.shape[0])
#     w_lo, w_hi = max(start[1],0), min(end[1], combined_map.shape[1])
#     center = (idx[0] - h_lo, idx[1] - w_lo)
#     mask = create_ball_mask(h_hi - h_lo, w_hi - w_lo, center=center, radius = dist)
#     combined_map[h_lo:h_hi, w_lo:w_hi][mask] = 1
#     # combined_map[tuple(idx)] = -1
#     combined_map[tuple(idx_beacon)] = -2

# sensor_locs
# (combined_map.T[10 - axis_mins[1]] > 0).sum()
import multirange as mr
from tqdm import tqdm
for y_idx in tqdm(range(1874066,4000000 + 1)):
    # y_idx = 2000000 #10
    y_idx_exp = np.array([y_idx] * len(sensor_locs))

    x_dist = detection_dists - np.abs((sensor_locs[:,1] - y_idx))
    valid = x_dist > 0
    x_lo = sensor_locs[:,0] - x_dist
    x_hi = sensor_locs[:,0] + x_dist

    # # For part 1 we need this, but part 2 doesnt since the distress beacon isnt a detected beacon
    # x_lo[np.all(beacon_locs == np.stack([x_lo, y_idx_exp], axis=-1), axis=-1)] += 1
    # x_hi[np.all(beacon_locs == np.stack([x_hi, y_idx_exp], axis=-1), axis=-1)] -= 1
    # _mr = list(mr.normalize_multi([range(_lo, _hi) for _lo, _hi in zip(x_lo[valid], x_hi[valid] + 1)]))
    # count = sum([len(r) for r in _mr])

    # Part 2
    _mr = list(mr.normalize_multi([range(_lo, _hi) for _lo, _hi in zip(x_lo[valid], x_hi[valid] + 1)]))
    intersect = list(mr.multi_intersection(_mr, [range(0,4000000 + 1)]))
    if len(intersect) > 1:
        assert(len(intersect) == 2)
        print('Found beacon:', (intersect[0].stop, y_idx))
        print('Tuning freq:', (intersect[0].stop) * 4000000 + y_idx)
        break

 35%|███▌      | 751340/2125935 [00:15<00:28, 47976.02it/s]

Found beacon: (2740279, 2625406)
Tuning freq: 10961118625406


In [226]:

# class MultiRange():
#     def __init__(self, ranges):
#         self.ranges_sorted = []
    
#     def update(self, value):
        
#         extended = False
#         for _range in self.ranges_sorted:
#             if   value[1] < _range[0] or _range[1] < value[0]: continue # disjoint
#             elif _range[0] <= value[0] and value[1] <= _range[1]: return # included
#             elif _range[0] <= value[0] and _range[1] <= value[1]: 

In [228]:
# !pip install multirange
# np.stack([x_lo[valid], x_hi[valid]], axis=-1)
# [len(r) for r in list(_mr)]
# count

26

In [191]:
# beacon_locs[:,0][valid] - x_lo[valid]
# x_dist
# valid
# detection_dists

In [206]:
# sensor_locs = np.array(sensor_locs)
# beacon_locs = np.array(beacon_locs)
# detection_dists = np.abs((beacon_locs - sensor_locs)).sum(axis=-1)
10 - axis_mins[1]

20

In [124]:
def print_map(combined_map):
    for row in combined_map:
        print(''.join(['.' if rowv <= 0 else '#' for rowv in row]))

# print_map([combined_map.T[10 - axis_mins[1]]])
print_map(combined_map.T)
# [combined_map.T[10 - axis_mins[1]]]

..........#..........................
.........###.........................
........#####........................
.......#######.......................
......#########.............#........
.....###########...........###.......
....#############.........#####......
...###############.......#######.....
..#################.....#########....
.###################.#.###########...
###################################..
.###################################.
..#################################..
...####################.##########...
....#############################....
.....###########################.....
......#########################......
.......#######################.......
........#######################......
.......#########################.....
......####.######################....
.....#################.###########...
......#############################..
.......#############################.
.......##############################
.......#############################.
.....#######

In [200]:
inputs = '''Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3'''


In [229]:
inputs = '''Sensor at x=2692921, y=2988627: closest beacon is at x=2453611, y=3029623
Sensor at x=1557973, y=1620482: closest beacon is at x=1908435, y=2403457
Sensor at x=278431, y=3878878: closest beacon is at x=-1050422, y=3218536
Sensor at x=1432037, y=3317707: closest beacon is at x=2453611, y=3029623
Sensor at x=3191434, y=3564121: closest beacon is at x=3420256, y=2939344
Sensor at x=3080887, y=2781756: closest beacon is at x=3420256, y=2939344
Sensor at x=3543287, y=3060807: closest beacon is at x=3420256, y=2939344
Sensor at x=2476158, y=3949016: closest beacon is at x=2453611, y=3029623
Sensor at x=3999769, y=3985671: closest beacon is at x=3420256, y=2939344
Sensor at x=2435331, y=2200565: closest beacon is at x=1908435, y=2403457
Sensor at x=3970047, y=2036397: closest beacon is at x=3691788, y=1874066
Sensor at x=2232167, y=2750817: closest beacon is at x=2453611, y=3029623
Sensor at x=157988, y=333826: closest beacon is at x=-1236383, y=477990
Sensor at x=1035254, y=2261267: closest beacon is at x=1908435, y=2403457
Sensor at x=1154009, y=888885: closest beacon is at x=1070922, y=-543463
Sensor at x=2704724, y=257848: closest beacon is at x=3428489, y=-741777
Sensor at x=3672526, y=2651153: closest beacon is at x=3420256, y=2939344
Sensor at x=2030614, y=2603134: closest beacon is at x=1908435, y=2403457
Sensor at x=2550448, y=2781018: closest beacon is at x=2453611, y=3029623
Sensor at x=3162759, y=2196461: closest beacon is at x=3691788, y=1874066
Sensor at x=463834, y=1709480: closest beacon is at x=-208427, y=2000000
Sensor at x=217427, y=2725325: closest beacon is at x=-208427, y=2000000
Sensor at x=3903198, y=945190: closest beacon is at x=3691788, y=1874066'''